<a href="https://colab.research.google.com/github/QueeneDelmarva/Thesis/blob/Debugging-Radix-Trie/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thesis Code**

## Importing Libraries

In [24]:
# Connect to Gdrive
from google.colab import drive

# Connect to Gsheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Preprocessing
import re
import nltk

# Edit Distance
import Levenshtein

# Generate Misspellings
import random
import string

# GUI
import tkinter as tk
import tkinter.font as tkFont

import string
from google.colab import auth
from google.auth import default

In [25]:
# !pip install python-Levenshtein


## Connecting to Gdrive

In [26]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Connecting to GSheets

In [27]:
# Define the scope and create the credentials using the file in your Google Drive
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/skripsi-394804-a98ea9715aa4.json', scope)

# Authenticate with gspread
client = gspread.authorize(creds)

# Open the Google Sheet by its title or URL
spreadsheet_key = '1RlZ8-XwwEueuCAq4nXGwq3ZmPDE-_0gP3PsroKeNFGo'
sheet = client.open_by_key(spreadsheet_key).sheet1

# Open the train_data Google Sheet by its title or URL for writing
train_data_sheet = client.open_by_key(spreadsheet_key).worksheet('train_data')
test_data_sheet = client.open_by_key(spreadsheet_key).worksheet('test_data')

# ## Read the data from the Gsheets
# # 1. Read individual cells
# cell_value = sheet.cell(3, 5).value
# print(cell_value)

# # 2. Read all values from the first worksheet
# data = sheet.get_all_values()

# # Print the data
# for row in data:
#     print(row)

## Preprocessing Data

In [28]:
# preprocess_dictonary = ['Noted', 'and', 'thanks', 'Please', 'as', 'follows', 'Copied']

def preprocess(word):
    # Separate words based on non-alphabetic characters
    words = re.findall(r'\b(?:[a-zA-Z0-9]+)\b', word)

    # Convert words to lowercase
    lowercase_words = [w.lower() for w in words]

    return lowercase_words

    # Add preprocess dictonary to the list of preprocessed words
    # preprocessed_words = lowercase_words + [g for g in preprocess_dictonary if g in word.lower()]

    # return preprocessed_words

### Radix Trie Node

In [29]:
# Defines a class representing a single node in the radix trie, holding information about its children and whether it marks the end of a word.
class RadixTrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

### Hashmap-based Trie Node

In [30]:
# Defines a class representing a single node in the radix trie, holding information about its children and whether it marks the end of a word.
class HashmapTrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False
        self.char = ""  # Add this line

### Radix Trie Function

In [71]:
# Defines a class that implements a radix trie data structure, allowing efficient storage and retrieval of a set of strings with common prefixes.
class RadixTrie:
    def __init__(self):
        self.root = RadixTrieNode()

    # Adds a word to the Radix Trie by creating or traversing the appropriate nodes for each character in the word.
    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = RadixTrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    # Finds all words in the Radix Trie that have the given prefix, returning them as a list of results.
    def search(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []  # Prefix not found, return an empty list
            node = node.children[char]
        return self.collect_words(node, prefix)

   # Recursively explores the Radix Trie's child nodes and appends the complete words to the results list when reaching the end of a word.
    def collect_words(self, node, prefix, words):
      # Base case: If the node is an end node, add the word to the list
      if node.is_end_of_word:
        words.append(prefix)

      # Recursively traverse all child nodes
      for char, child in node.children.items():
        self.collect_words(child, prefix + char, words)  # Use 'child' instead of 'child_node'

      return words

    def get_all_words(self):
      words = []
      self.collect_words(self.root, "", words)
      return words

    def visualize(self, node, prefix, row_index, column_index):
      if node.is_end_of_word:
        output = prefix + " end of word"
        print(output)
        train_data_sheet.update_cell(row_index, column_index, output)
        row_index += 1

      for char, child in node.children.items():
        if char != prefix:
            output = prefix + "| " + char + " -->"
            print(output)
            train_data_sheet.update_cell(row_index, column_index, output)
            self.visualize(child, prefix + "| ", row_index + 1, column_index)


### Hashmap-based Trie

In [32]:
# Define a class represents a trie data structure that uses a hashmap to store children nodes, allowing efficient storage and retrieval of a set of strings with common prefixes.
class HashmapTrie:
    def __init__(self):
        self.root = HashmapTrieNode()

# Adds a word to the trie by traversing and creating nodes for each character in the word and marking the last node as the end of the word.
    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = HashmapTrieNode()
            node = node.children[char]
        node.is_end_of_word = True

# Checks if a given prefix exists in the trie by traversing the nodes based on the characters of the prefix and returns whether the last node reached marks the end of a word.
    def search(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return []  # Prefix not found, return an empty list
            node = node.children[char]
        return self.collect_words(node, prefix)

# Recursively traverses its child nodes and appends the complete words to the results list when reaching the end of a word.
    def collect_words(self, node, current_prefix):
        results = []
        if node.is_end_of_word:
            results.append(current_prefix)
        for char, child in node.children.items():
            word = current_prefix + char
            results.extend(self.collect_words(child, word))
        return results

# Returns all words stored in the trie by collecting words
    def get_all_words(self):
        return self.collect_words(self.root, "")

### Autocorrect with Levenshtein Edit Distance

In [33]:
class Autocorrect:
    def __init__(self, trie):
        self.trie = trie

    def correct(self, word, threshold=0.9):  # Adjust threshold as needed
        word = word.lower()
        suggestions = []

        # Traverse the trie to find similar words
        similar_words = self.trie.search_similar(word)

        # Calculate Jaro-Winkler distance and filter based on threshold
        for similar_word in similar_words:
            similarity = distance.get_jaro_distance(word, similar_word)
            if similarity >= threshold:
                suggestions.append(similar_word)

        if suggestions:
            return suggestions[0]  # Return the first suggestion
        else:
            return None

### Generate Misspellings

In [34]:
class MisspelledWordGenerator:
    def __init__(self, threshold=0.2):
        self.threshold = threshold

    def generate(self, word):
        misspelled_word = []
        for char in word:
            if random.random() < self.threshold:
                misspelled_word.append(random.choice(string.ascii_lowercase))
            else:
                misspelled_word.append(char)
        return ''.join(misspelled_word)

### Weight Parameters

In [35]:
class WeightEvaluator:
    def __init__(self, trie_structure, weight_ranges):
        self.trie_structure = trie_structure
        self.weight_ranges = weight_ranges

    def evaluate_weights(self, threshold):
        print("Weightage | Accuracy | MMR")

        for w1, w2, w3 in self.weight_ranges:
            accuracy, mmr = self.trie_structure.evaluate_performance(threshold, w1, w2, w3)
            print(f"{w1:.2f}, {w2:.2f}, {w3:.2f} | {accuracy:.3f} | {mmr:.3f}")

### Performance Evaluation

In [36]:
class EvaluationPerformance:
    def __init__(self, trie_structure, weight_settings):
        self.trie_structure = trie_structure
        self.w1, self.w2, self.w3 = weight_settings

    def calculate_prefix_score(self, word, dictionary):
    # Print the data types of word and terms in dictionary
        # print("Word data type:", type(word))
        # print("Term data type in dictionary:", [type(term) for term in dictionary])
        # Calculate and return the prefix score for the word and dictionary
        prefix_score = 0
        for term in dictionary:
            if term.startswith(word):
                prefix_score += len(word)
        return prefix_score

    def calculate_suffix_score(self, word, dictionary):
        # Calculate and return the suffix score for the word and dictionary
        suffix_score = 0
        for term in dictionary:
            reversed_term = term[::-1]
            reversed_word = word[::-1]
            if reversed_term.startswith(reversed_word):
                suffix_score += len(reversed_word)
        return suffix_score

    def calculate_similarity_score(self, word, term):
      lcs = self.LongestCommonSubsequence(word, term)
      similarity_score = (2 * lcs) / (len(word) + len(term))
      # print(f"Word: {word}, Term: {term}, LCS: {lcs}, Similarity Score: {similarity_score}")
      return similarity_score


    def LongestCommonSubsequence(self, text1, text2):
        m, n = len(text1), len(text2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if text1[i - 1] == text2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])

        return dp[m][n]

    def evaluate(self, test_data):
      results = []
      for query, correct_term in test_data:
            # print("Query:", query, "Correct Term:", correct_term)
            dictionary = self.trie_structure.get_all_words()
            # print("Dictionary:", dictionary)
            # print("Correct Term:", correct_term)
            prefix_score = self.calculate_prefix_score(query, dictionary)
            suffix_score = self.calculate_suffix_score(query, dictionary)
            similarity_score = self.calculate_similarity_score(query, correct_term)

            # Calculate the overall score based on weights w1, w2, and w3
            overall_score = (self.w1 * prefix_score) + (self.w2 * suffix_score) + (self.w3 * similarity_score)

            results.append((query, correct_term, overall_score, prefix_score, suffix_score, similarity_score))

            # print(f"Query: {query}, Correct Term: {correct_term}")
            # print(f"Prefix Score: {prefix_score}, Suffix Score: {suffix_score}, Similarity Score: {similarity_score}")

      return results

### **Main Function**

In [73]:
def main():
    # Create a Radix Trie
    trie = RadixTrie()
    words = ["and", "acam", "as"]
    for word in words:
        trie.insert(word)

    # Visualize the Radix Trie for each letter
    for letter in trie.root.children:
        print(letter)
        output = letter + " -->"
        column_index = 3  # Initialize column_index here
        row_index = 2     # Initialize row_index here
        print(output)
        train_data_sheet.update_cell(row_index, column_index, output)
        trie.visualize(trie.root.children[letter], "", row_index + 1, column_index)

if __name__ == "__main__":
    main()

a
a -->
| n -->
| | d -->
| |  end of word
| c -->
| | a -->
| | | m -->
| | |  end of word
| s -->
|  end of word


In [65]:
def main():
    radix_trie = RadixTrie()

    # Add your train_data and test_data here
    train_data = ["CAT", "CAR", "CORN", "CORT", "CORNY"]

    # Populate trie with terms from train_data
    for term in train_data:
        radix_trie.insert(term)

    trie.visualize()

    # test_data = [
    #     ("PgeprjcessedgData", "Processed Data"),
    #     ("dtae", "date"),
    #     ("contknt", "contact"),
    #     ("transport", "trdnsyort"),
    #     ("wad", "wed"),
    #     # ... add more test cases
    # ]

    # radix_trie.list_all_nodes()  # List all nodes
    # radix_trie.print_tree()  # Print tree structure
    # radix_trie.print_statistics()  # Print statistics

    # hashmap_trie = HashmapTrie()

    # # Fixed weight settings (suffix_weight, similarity_weight)
    # w2 = 0.20
    # w3 = 1.1

    # # print("Weight Study Results:")
    # # print("{:<10} | {:<10} | {:<10}".format("w1", "Accuracy", "MMR"))
    # # print("-" * 30)

    # # Test data for evaluation
    # test_data = [
    #     ("PgeprjcessedgData", "Processed Data"),
    #     ("dtae", "date"),
    #     ("contknt", "contact"),
    #     ("transport", "trdnsyort"),
    #     ("wad", "wed"),
    #     ("projekt", "project"),
    #     # ... add more test cases
    # ]

    # # Populate trie with terms from train_data
    # train_data = ["Processed Data", "date", "contact", "trdnsyort", "wed"]
    # for term in train_data:
    #     radix_trie.insert(term)  # Or hashmap_trie.add(term)
    #     trie_words = radix_trie.get_all_words()  # Or hashmap_trie.get_all_words()
    #     # print("Trie words:", trie_words)

    # # Varying w1 and keeping w2, w3 fixed
    # for w1 in [0.15, 0.20, 0.25, 0.30, 0.35]:
    #     evaluator = EvaluationPerformance(radix_trie, (w1, w2, w3))
    #     evaluation_results = evaluator.evaluate(test_data)

    #     # Extract MMR values from evaluation_results
    #     mmr_values = [result[5] for result in evaluation_results]  # Index 5 corresponds to the MMR value

    #     # Calculate average MMR value
    #     average_mmr = sum(mmr_values) / len(mmr_values)

    #     # print("Weight Settings: (w1 = {}, w2 = {}, w3 = {})".format(w1, w2, w3))
    #     # print("{:<10} | {:<10.3f}".format("w1", average_mmr))
    #     # print("-" * 20)

if __name__ == "__main__":
    main()

NameError: ignored

In [ ]:
# def main():
#     # Create instances of the trie structures and the autocorrect class
#     radix_trie = RadixTrie()
#     hashmap_trie = HashmapTrie()

#     # Fixed weight settings (suffix_weight, similarity_weight)
#     w2 = 0.20
#     w3 = 1.1

#     print("Weight Study Results:")
#     print("{:<10} | {:<10} | {:<10}".format("w1", "Accuracy", "MMR"))
#     print("-" * 30)

#     # Test data for evaluation
#     test_data = [
#         ("PgeprjcessedgData", "ProcessedData"),
#         ("dtae", "date"),
#         ("contknt", "contact"),
#         ("transport", "trdnsyort"),
#         ("wad", "wed"),
#         # ... add more test cases
#     ]

#     # Varying w1 and keeping w2, w3 fixed
#     # Varying w1 and keeping w2, w3 fixed
#     for w1 in [0.15, 0.20, 0.25, 0.30, 0.35]:
#       evaluator = EvaluationPerformance(radix_trie, (w1, w2, w3))
#       evaluation_results = evaluator.evaluate(test_data)

#       mmr_values = [result[2] for result in evaluation_results]  # Extract MMR value from result tuple

#       average_mmr = sum(mmr_values) / len(mmr_values)

#       print("Weight Settings: (w1 = {}, w2 = {}, w3 = {})".format(w1, w2, w3))
#       print("{:<10} | {:<10}".format("w1", "MMR"))
#       print("-" * 20)

#       for result in evaluation_results:
#         query, correct_term, overall_score, prefix_score, suffix_score, similarity_score = result
#         # print(f"Query: {query}, Correct Term: {correct_term}, Prefix Score: {prefix_score}, Suffix Score: {suffix_score}, Similarity Score: {similarity_score}, Overall Score: {overall_score}")

#         print("{:<10} | {:<10.3f}".format(w1, result[2]))  # Use result[2] for MMR value

# if __name__ == "__main__":
#     main()


# if __name__ == "__main__":
#     main()


In [ ]:
    # def evaluate(self, test_data):
    #     accuracy_values = []
    #     mmr_values = []
    #     results = []
    #     threshold = 0.7  # Set the similarity score threshold here

    #     for query, correct_term in test_data:
    #         dictionary = self.trie_structure.get_all_words()
    #         prefix_score = self.calculate_prefix_score(query, dictionary)
    #         suffix_score = self.calculate_suffix_score(query, dictionary)
    #         similarity_score = self.calculate_similarity_score(query, correct_term)

    #         # Calculate the overall score based on weights w1, w2, and w3
    #         overall_score = (self.w1 * prefix_score) + (self.w2 * suffix_score) + (self.w3 * similarity_score)

    #         # Calculate the predicted term based on similarity score threshold
    #         if similarity_score >= threshold:
    #             predicted_term = correct_term  # Use correct_term when similarity score is high
    #         else:
    #             predicted_term = self.trie_structure.find_most_similar_term(query)  # Replace with your method

    #         accuracy = 1 if predicted_term == correct_term else 0
    #         mmr = overall_score  # Use overall score as MMR for now

    #         accuracy_values.append(accuracy)
    #         mmr_values.append(mmr)

    #     return accuracy_values, mmr_values

    # def evaluate(self, test_data):
    #   results = []
    #   threshold = 0.7  # Set the similarity score threshold here

    #   for query, correct_term in test_data:
    #     dictionary = self.trie_structure.get_all_words()
    #     prefix_score = self.calculate_prefix_score(query, dictionary)
    #     suffix_score = self.calculate_suffix_score(query, dictionary)
    #     similarity_score = self.calculate_similarity_score(query, correct_term)

    #     # Calculate the predicted term based on similarity score threshold
    #     if similarity_score >= threshold:
    #         predicted_term = correct_term  # Use correct_term when similarity score is high
    #     else:
    #         predicted_term = query  # Use query when similarity score is low

    #     # Calculate the overall score based on weights w1, w2, and w3
    #     overall_score = (self.w1 * prefix_score) + (self.w2 * suffix_score) + (self.w3 * similarity_score)

    #     results.append((query, correct_term, predicted_term, overall_score))

    #   return results

# # For debugging purposes, print the query, correct term, and similarity score
  # print("Query:", query)
  # print("Correct Term:", correct_term)
  # print("Similarity Score:", similarity_score)

# Output
# Query: PgeprjcessedgData
# Correct Term: ProcessedData
# Similarity Score: 0.8
# Query: dtae
# Correct Term: date
# Similarity Score: 0.75
# Query: contknt
# Correct Term: contact
# Similarity Score: 0.7142857142857143